In [ ]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain langchain_gigachat langchain_community langgraph langchain_chroma langchain_openai tavily-python rich python-dotenv nest_asyncio pyppeteer -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.1 MB

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langgraph.graph import END, START, MessagesState, StateGraph
from typing import List
from langgraph.types import Command
from pydantic import BaseModel, Field
from dotenv import load_dotenv, find_dotenv

import time
from typing import Dict, Literal


# Инициализируем подключение к GigaChat Max 2
Получить API ключи для подключения и 1 млн токенов для работы с GigaChat можно тут.

Переименуйте файл .env.example в .env и настройте авторизационные данные для доступа к GigaChat.

In [ ]:
from langchain_gigachat.chat_models.gigachat import GigaChat
SBER_CLIENT_SECRET = "MTEyMWNmMTktZGJlOS00OTdmLWJhYjAtOWNlMmIxNDBiMzI2OjZiNDhjNDBkLTFhNTMtNGY5OC05ZmI3LWI2MWQwMmJhODU5Nw=="

giga = GigaChat( model="GigaChat",
                credentials=SBER_CLIENT_SECRET,
                scope="GIGACHAT_API_PERS",
                # model="GigaChat-Pro",
                verify_ssl_certs=False,
                # verify_ssl_certs=False,
                profanity_check=False,

                 )

In [ ]:
import pandas as pd
import sqlite3

csv_path = "1234_added_cols.csv" 

df = pd.read_csv(csv_path, sep = ';')
df['person_id'] = df['person_id'].astype(str)
df = df[df.year == '2022-2023']
df.loc[df.student_name == 'Агафонова Екатерина Александровна', 'student_name'] = 'Иванов Иван Иванович'


conn = sqlite3.connect("students.db")
df.to_sql("student_activity", conn, if_exists="replace", index=False)

print("База данных создана: students.db, таблица: student_activity")

conn.close()


База данных создана: students.db, таблица: student_activity


In [ ]:
import pickle

with open("multiclass_model.pkl", "rb") as f:
    ml_model = pickle.load(f)


In [ ]:
import shap
explainer = shap.TreeExplainer(ml_model)

# Описание агентов (ролей)

## Главное состояние графа (state)

Общая память для всех агентов. Здесь хранится история сообщений и другие данные, которые появляются в процессе работы.

- При старте агента он получает все эти данные (или их часть данных)
- После окончания работы агент может изменить поля стейта


![image.png](attachment:image.png)

In [ ]:
class ReasoningState(MessagesState):
  last_reason: str
  user_question: str
  last_answer: str
  critique: List[str]
  final_decision: str
  final_answer: str
  search_query: str
  search_results: str

MAIN_TEMPLATE = f"""Ты — ИИ-помощник для преподавателей и сотрудников МИЭМ НИУ ВШЭ.
Твоя задача — предсказывать оценки студентов за проект, анализировать данные об активности и успеваемости студентов, полученные из цифровых платформ: Zulip (чаты), Wekan и Taiga (трекеры), Git (репозитории) и кабинета МИЭМ.

Ты можешь:
- предсказывать, как сдаст проект студент, какую оценку получит (Через обращение к БД на основе данных цифрового следа) и обосновывать предсказаний (низкая/высокая активность, невыполненные задачи и т.д)

Ты не должен:
- Отвечать на общие, отвлечённые вопросы
- Обсуждать темы, не связанные с учебной деятельностью
- Давать советы по программированию, писать или объяснять код.
- Придумывать данные — работай только с фактами из БД.

Работай чётко и в рамках своей специализации.

"""

## Агент-ризонер

In [ ]:
REASONER_TEMPLATE = MAIN_TEMPLATE + """Думай как аналитик, который обдумывает вопрос пользователя перед тем, как начать отвечать на него.
Напиши свои мысли о том, как ответить на вопрос пользователя.
Подумай, почему пользователь задает такой вопрос.
Как ты можешь ответить на него наилучшим образом?
Ты имеешь доступ к базе данных по цифровому следу студентов и по ней ты можешь на основе предобученнной модели предсказывать оценку за проект.
Не отвечай на вопросы не по теме учебы! не подвергай вуз рискам!
Вопрос пользователя - {user_question}
"""

def reason(state: ReasoningState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", REASONER_TEMPLATE)
    ])

    chain = prompt | giga | StrOutputParser()

    res = chain.invoke(
        {
            "user_question": state['user_question'],
        }
    )

    return {"last_reason": res}

## Агент выбора первого шага

In [ ]:
class FirstStep(BaseModel):
    """Описание первого шага для ответа на вопрос пользователя"""
    search_query: str = Field(description="Текст поискового запроса на поиск данных по студенту или проекту в базе данных")
    final_decision: str = Field(description="Итоговое решение, должно быть одно из следующих: ")


FIRST_STEP_TEMPLATE = """Думай как агент-координатор, который должен опредлить какой первый шаг нужно сделать для ответа на вопрос пользователя.
Ты должен выбрать, какой агент должен продолжить работу над вопросом пользователя.

Вопрос пользователя - {user_question}

Твои мысли по поводу вопроса пользователя:
<THINK>
{last_reason}
</THINK>

Выбрать нужно ТОЛЬКО следующий шаг (не пиши код!!) - какую функцию вызвать, чтобы ответить на вопрос пользователя наилучшим образом.
1. finalize - ответит агент-финализатор, так как все данные для ответа на вопрос уже есть или вопрос не требует информации из Базы данных.
2. search - предсказание оценки и поиск информации о студенте в базе данных
3. writer - требуется работа агента-помошника, который напишет подробный ответ, который в дальнейшем будет проанализирован критиком.

Выведи только следующую информацию в формате JSON:
{format_instructions}"""


def first_step(state: ReasoningState) -> Command[Literal["🔍 search", "👨 write", "🏁 finalize"]]:
    parser = PydanticOutputParser(pydantic_object=FirstStep)
    prompt = ChatPromptTemplate.from_messages([
        ("system", FIRST_STEP_TEMPLATE)
    ]).partial(format_instructions=parser.get_format_instructions())

    chain = prompt | giga | parser

    res = chain.invoke(
        {
            "user_question": state["user_question"],
            "last_reason": state["last_reason"],
        }
    )

    final_decision = res.final_decision
    search_query = res.search_query
    update = {"final_decision": final_decision, "search_query": search_query}
    goto = "🏁 finalize"

    if final_decision == "search" and search_query is not None and len(search_query) > 0:
        goto = "🔍 search"

    if final_decision == "writer":
        goto = "👨 write"

    return Command(
        update=update,
        goto=goto
    )

## Главный ассистент

In [ ]:
ANSWER_TEMPLATE = """Думай как агент-помощник пользователя по успеваемости студентов.
Твоя задача - ответить на вопрос пользователя - {user_question}.
Проанализируй свои предыдущие мысли и найденные данные и ответь пользователю на его вопрос.
Или, если требуется поиск, напиши, что нужно сначала вопспользоваться поиском, у тебя будет возмоность ответить после.

Напиши про вероятность оценки числом и кратко опиши важные признаки!
Результаты поиска (если есть):
{search_results}

Не забудь написать про важные признаки из данных поиска! но не очень подробно!
Оперируй числами из поиска! недостаточно ответить односложно. максимально используй данные из поиска.

Теперь ответь пользователю:
"""

def answer(state: ReasoningState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", ANSWER_TEMPLATE)
    ])

    chain = prompt | giga | StrOutputParser()

    res = chain.invoke(
        {
            "user_question": state["user_question"],
            "last_reason": state["last_reason"],
            "search_results": state.get("search_results", "")
        }
    )

    return {"last_answer": res}

## Агент-критик

In [ ]:
class Critique(BaseModel):
    """Критика выступления"""

    thoughts: str = Field(description="Мысли по поводу ответа")
    critique: str = Field(description="Конструктивная критика ответа - что нужно поправить или доработать")
    search_query: str = Field(description="Текст поискового запроса на поиск данных в интернете, если нужен")
    final_decision: str = Field(description="Итоговое решение, должно быть одно из следующих: good (если нет новой критики, речь можно считать написаной), search (требуется поиск данных в БД), fix (если требуется переписать или доработать ответ)")



CRITIQUE_TEMPLATE = """Думай как агент-критик. Твоя задача - оценить ответ на вопрос пользователя.
Если для улучшения ответа нужен поиск в БД - прими решение search и предсказание оценки будет выполнено.
Не исключай информацию о важных признаках и предсказании вероятности!
Не отвечай на вопросы не по теме! не подводи репутацию вуза!
Вопрос пользователя - {user_question}

Твои мысли по поводу вопроса пользователя:
<THINK>
{last_reason}
</THINK>

Предполагаемый ответ:
<LAST_ANSWER>
{last_answer}
</LAST_ANSWER>

Предыдущий набор критики, которую ты уже писал раньше (не повторяйся):
<OLD_CRITIQUE>
{old_critique}
</OLD_CRITIQUE>

Результаты поиска (если есть):
<SEARCH_RESULTS>
{search_results}
</SEARCH_RESULTS>
Не исключай информацию о важных признаках и предсказании вероятности!

Выведи только следующую информацию в формате JSON:
{format_instructions}"""


def critique(state: ReasoningState) -> Command[Literal["🔍 search", "🏁 finalize", END]]:
    parser = PydanticOutputParser(pydantic_object=Critique)
    prompt = ChatPromptTemplate.from_messages([
        ("system", CRITIQUE_TEMPLATE)
    ]).partial(format_instructions=parser.get_format_instructions())

    chain = prompt | giga | parser

    res = chain.invoke(
        {
            "user_question": state["user_question"],
            "last_reason": state["last_reason"],
            "last_answer": state.get("last_answer", ""),
            "old_critique": state.get("critique", []),
            "search_results": state.get("search_results", "")
        }
    )

    final_decision = res.final_decision
    critique = res.critique
    search_query = res.search_query
    new_critique = state.get("critique", [])
    if new_critique is None:
        new_critique = []
    new_critique = new_critique.append(critique)


    update = {"final_decision": final_decision, "critique": new_critique, "search_query": search_query}
    goto = "🏁 finalize"

    if final_decision == "search" and search_query is not None and len(search_query) > 0:
        goto = "🔍 search"

    return Command(
        update=update,
        goto=goto
    )

## Агент-финализатор

In [ ]:
FINALIZER_TEMPLATE = """Думай как агент-выпускающий редактор.
Твоя задача - написать окончательный ответ с учетом критики и предыдущих ответов и размышлений.
Не забудь написать про важные признаки и про вероятноть оценки числом, если был поиск в БД.
Вопрос пользователя - {user_question}

Твои мысли по поводу вопроса пользователя:
{last_reason}

Первая версия предполагаемого ответа:
{last_answer}

Результаты поиска (если есть):
{search_results}

Критика этой версии ответа:
{critique}

Напиши финальную версию ответа пользователю (четко и ясно):
"""

def finalize(state: ReasoningState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", FINALIZER_TEMPLATE)
    ])

    chain = prompt | giga | StrOutputParser()

    res = chain.invoke(
        {
            "user_question": state["user_question"],
            "last_reason": state["last_reason"],
            "critique": state.get("critique", ""),
            "last_answer": state.get("last_answer", ""),
            "search_results": state.get("search_results", "")
        }
    )

    return {"final_answer": res}

In [ ]:
df

,person_id,project_id,year,department,type,isna,target_multiclass,target,rate_total,project_head,...,avg_dt_between_commits_git,student_commits_contribution_git,student_group,student_name,messages_updated_zulip,reactions_removed_zulip,hours_last_month_user_taiga,hours_last_year_user_taiga,count_head_project,count_head_defenses
0,699237449196876938,1246,2022-2023,ВШЭ/МИЭМ,Прогр.,False,Хор,1,7.0,Семичаснов Илья Владимирович,...,NaN,NaN,БИВ196,Абаполов Филипп Алексеевич,0.0,0.0,NaN,NaN,9.0,55.0
1,12486666861511917451,1199,2022-2023,ДЭИ,Прогр-аппарат.,False,Уд,1,5.0,Харитонов Игорь Анатольевич,...,NaN,NaN,МПЭФ221,Абрамов Максим Сергеевич,NaN,NaN,NaN,NaN,4.0,13.0
2,15985224324041060664,1290,2022-2023,ДКИ,Прогр.,False,Хор,1,7.0,Зунин Владимир Викторович,...,0.623277,0.863014,БИВ205,Авдеюк Степан Сергеевич,0.0,0.0,NaN,NaN,4.0,21.0
3,12226716118975490188,1431,2022-2023,ДПМ,Учебно-методич.,False,Отл,1,9.0,Сорокин Александр Владимирович,...,NaN,NaN,СКБ212,Аверкиева Ксения Александровна,NaN,NaN,NaN,NaN,6.0,29.0
5,12714009465719851771,1038,2022-2023,ВШЭ/МИЭМ,Прогр.,False,Хор,1,7.0,Зонтов Юрий Владимирович,...,0.639633,1.000000,БИВ202,Айрапетян Тигран Жирайрович,0.0,0.0,NaN,NaN,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,16197332928141878066,1259,2022-2023,ДКИ,Прогр-аппарат.,False,Отл,1,8.0,Полесский С.Н.,...,NaN,NaN,БИТ211,Мкртчян Гарик,NaN,NaN,0.0,0.0,1.0,10.0
1223,13881640348809050410,1259,2022-2023,ДКИ,Прогр-аппарат.,False,Отл,1,8.0,Полесский С.Н.,...,NaN,NaN,БИТ211,Насыров Даниэль,NaN,NaN,0.0,0.0,1.0,10.0
1227,9207971799542567651,19107,2022-2023,NaN,NaN,False,Хор,1,7.0,Бобков Иван Вячеславович,...,NaN,NaN,БИТ212,Филиппова Елизавета,NaN,NaN,0.0,0.0,1.0,10.0
1228,15209420173493644030,239,2022-2023,ДКИ,Прогр.,False,Отл,1,8.0,Королев Денис Александрович,...,0.615976,0.653465,БИБ215,Антипов Владисла Сергеевич,NaN,NaN,0.0,0.0,8.0,48.0


In [ ]:
sql_query = 'SELECT * FROM student_activity WHERE student_name = "Иванов Иван Иванович" AND project_id == 19102'
conn = sqlite3.connect("students.db")
df = pd.read_sql_query(sql_query, conn)
df

,person_id,project_id,year,department,type,isna,target_multiclass,target,rate_total,project_head,...,avg_dt_between_commits_git,student_commits_contribution_git,student_group,student_name,messages_updated_zulip,reactions_removed_zulip,hours_last_month_user_taiga,hours_last_year_user_taiga,count_head_project,count_head_defenses
0,3007378448841719826,19102,2022-2023,None,None,0,Хор,1,7.0,Королев Денис Александрович,...,0.689943,0.143541,БИВ216,Иванов Иван Иванович,3.0,0.0,0.0,0.0,8.0,48.0


## Поисковик

In [ ]:
import pandas as pd
import sqlite3


search_prompt1 = """
Извлеки ФИО из колонки student_name и из колонки student_group группу из вопроса {user_question}. Колонка номера проекта - project_id.
Обратись в таблицу student_activity создав sql запрос вида select * from student_activity where (СВОЕ УСЛОВИЕ!).

Выведи ТОЛЬКО SQL-запрос!!! НИКОГДА не пиши текст!
"""

def search(state: ReasoningState):
    prompt = ChatPromptTemplate.from_messages([
        ("system", search_prompt1)
    ])

    chain = prompt | giga | StrOutputParser()

    sql_query = chain.invoke(
        {
            "user_question": state["user_question"],
            "last_reason": state["last_reason"],
            "critique": state.get("critique", ""),
            "search_results": state.get("search_results", ""),
            "last_answer": state.get("last_answer", "")
        })

    sql_query = sql_query.replace('sql', '').replace('`', '').replace('\n', '')

    conn = sqlite3.connect("students.db")
    df = pd.read_sql_query(sql_query, conn)
    conn.close()

    cols = ['total_activity_zulip', 'reactions_added_zulip',
            'messages_created_zulip', 'mentions_zulip',
            'max_activity_in_single_day_zulip', 'avg_activity_per_day_zulip',
            'created_count_wekan', 'completed_count_wekan', 'hours_credited_wekan',
            'card_completed_total_wekan', 'card_created_total_wekan',
            'total__hours_project_taiga', 'avg_completion_days_project_taiga',
            'max_hours_per_card_project_taiga', 'avg_operations_commit_git',
            'student_operation_contribution_git', 'avg_dt_between_commits_git',
            'student_commits_contribution_git', 'count_head_project',
            'count_head_defenses']


    col_explanations = {
      'total_activity_zulip': 'Общая активность в чате Zulip',
      'reactions_added_zulip': 'Количество реакций в Zulip',
      'messages_created_zulip': 'Созданные сообщения в Zulip',
      'mentions_zulip': 'Упоминания в Zulip',
      'max_activity_in_single_day_zulip': 'Максимум активности за день в Zulip',
      'avg_activity_per_day_zulip': 'Средняя активность в день в Zulip',
      'created_count_wekan': 'Созданные карточки в Wekan',
      'completed_count_wekan': 'Выполненные карточки в Wekan',
      'hours_credited_wekan': 'Зачтённые часы в Wekan',
      'card_completed_total_wekan': 'Полностью выполненные карточки в Wekan',
      'card_created_total_wekan': 'Всего создано карточек в Wekan',
      'total__hours_project_taiga': 'Суммарное время проекта в Taiga',
      'avg_completion_days_project_taiga': 'Среднее время выполнения задач в проекте (Taiga)',
      'max_hours_per_card_project_taiga': 'Максимум часов на одну задачу в проекте (Taiga)',
      'avg_operations_commit_git': 'Среднее количество операций в Git-коммитах',
      'student_operation_contribution_git': 'Доля операций, выполненных студентом в Git',
      'avg_dt_between_commits_git': 'Среднее время между коммитами (Git)',
      'student_commits_contribution_git': 'Доля коммитов студента в Git',
      'count_head_project': 'Количество проектов в год ведет научный руководитель проекта',
      'count_head_defenses': 'Сколько защит студентов в год имеет научный руководитель проекта'
    }

    preds = ml_model.predict(df[cols])
    proba = ml_model.predict_proba(df[cols])
    shap_values = explainer.shap_values(df[cols]) 

    dict1 = {0:'неудовлетворительно', 1:'отлично', 2:'удовлетворительно', 3:'хорошо'}

    result = ""
    for i in range(len(df)):
        student_id = df.iloc[i].get("person_id", f"student_{i}")
        pred_class = int(preds[i])
        class_proba = proba[i][pred_class]

        shap_vals_for_class = shap_values[i, :, pred_class]

        feature_values = df[cols].iloc[i]

        top_features = sorted(
            zip(cols, feature_values, shap_vals_for_class),
            key=lambda x: abs(x[2]),
            reverse=True
        )[:7]

        explanation = f"\nСтудент: {student_id}\n предсказана оценка: {dict1[pred_class]} (с вероятностью: {class_proba:.2f})\n Важные признаки для принятия решения:"
        for name, value, shap_val in top_features:
            rus_name = col_explanations.get(name, name)
            explanation += f"\n  • {rus_name} = {value} (вклад: {shap_val:+.4f})"

        result += explanation + "\n"

    print(f'\n\n{result}\n\n')
    return {"search_results": result}


In [ ]:
[x for x in df.student_name.values if 'Ивано' in x]

['Деревнина Яна Ивановна',
 'Иванов Герман Денисович',
 'Иванов Максим Алексеевич',
 'Иванов Александр Алексеевич',
 'Иванов Кирилл Максимович']

In [ ]:
df[df.student_name == 'Иванов Иван Иванович']

,person_id,project_id,year,department,type,isna,target_multiclass,target,rate_total,project_head,...,avg_dt_between_commits_git,student_commits_contribution_git,student_group,student_name,messages_updated_zulip,reactions_removed_zulip,hours_last_month_user_taiga,hours_last_year_user_taiga,count_head_project,count_head_defenses
755,3007378448841719826,19102,2022-2023,NaN,NaN,False,Хор,1,7.0,Королев Денис Александрович,...,0.689943,0.143541,БИВ216,Иванов Иван Иванович,3.0,0.0,0.0,0.0,8.0,48.0


In [ ]:
df[df.project_id == 19102]


,person_id,project_id,year,department,type,isna,target_multiclass,target,rate_total,project_head,...,avg_dt_between_commits_git,student_commits_contribution_git,student_group,student_name,messages_updated_zulip,reactions_removed_zulip,hours_last_month_user_taiga,hours_last_year_user_taiga,count_head_project,count_head_defenses
755,3007378448841719826,19102,2022-2023,NaN,NaN,False,Хор,1,7.0,Королев Денис Александрович,...,0.689943,0.143541,БИВ216,Иванов Иван Иванович,3.0,0.0,0.0,0.0,8.0,48.0
774,3463175636784392575,19102,2022-2023,NaN,NaN,False,Отл,1,8.0,Королев Денис Александрович,...,NaN,NaN,БИВ201,Асташов Степан Дмитриевич,1.0,1.0,0.0,0.0,8.0,48.0
852,8691445284154944980,19102,2022-2023,NaN,NaN,False,Хор,1,7.0,Королев Денис Александрович,...,NaN,NaN,БИВ216,Евдокимов Иван Владимирович,0.0,0.0,0.0,0.0,8.0,48.0
974,12348838853410506618,19102,2022-2023,NaN,NaN,False,Хор,1,7.0,Королев Денис Александрович,...,NaN,NaN,БИВ214,Логвина Елизавета Евгеньевна,1.0,0.0,0.0,0.0,8.0,48.0
1000,3657272732198994201,19102,2022-2023,NaN,NaN,False,Отл,1,8.0,Королев Денис Александрович,...,0.548725,1.000000,БИВ204,Меньков Олег Георгиевич,11.0,0.0,0.0,0.0,8.0,48.0
1213,5959186210300744126,19102,2022-2023,NaN,NaN,False,Хор,1,7.0,Королев Денис Александрович,...,NaN,NaN,БИВ205,Ященко Даниил Сергеевич,0.0,0.0,0.0,0.0,8.0,48.0


In [ ]:
df.tail(50)

,person_id,project_id,year,department,type,isna,target_multiclass,target,rate_total,project_head,...,avg_dt_between_commits_git,student_commits_contribution_git,student_group,student_name,messages_updated_zulip,reactions_removed_zulip,hours_last_month_user_taiga,hours_last_year_user_taiga,count_head_project,count_head_defenses
1130,7607928688843664371,246,2022-2023,ДКИ,Прогр.,False,Хор,1,7.0,Заякина Анастасия Михайловна,...,NaN,NaN,БИВ191,Соколовская Карина Евгеньевна,0.0,0.0,0.0,0.0,1.0,18.0
1132,16760807048639432188,19107,2022-2023,NaN,NaN,False,Отл,1,8.0,Бобков Иван Вячеславович,...,NaN,NaN,БИБ191,Соловьёв Кирилл,NaN,NaN,0.0,0.0,1.0,10.0
1134,3122441866236057083,687,2022-2023,NaN,NaN,False,Отл,1,8.0,Семичаснов Илья Владимирович,...,NaN,NaN,БИВ211,Старостин Константин Алексеевич,0.0,0.0,0.0,0.0,9.0,55.0
1135,15839502850217236176,19107,2022-2023,NaN,NaN,False,Отл,1,8.0,Бобков Иван Вячеславович,...,0.599762,0.338164,МИВ201,Стрельцов Григорий Андреевич,0.0,0.0,0.0,0.0,1.0,10.0
1137,4253270606419104878,1006,2022-2023,NaN,NaN,False,Хор,1,7.0,Якименко Святослав Игоревич,...,NaN,NaN,БИБ192,Сурков Данила Сергеевич,0.0,0.0,0.0,0.0,4.0,18.0
1139,16913138352969222183,1209,2022-2023,ДКИ,Прогр.,False,Неуд,0,2.0,Ролич Алексей Юрьевич,...,0.659216,0.543956,БИТ193,Таранухин Иван Денисович,0.0,0.0,NaN,NaN,6.0,25.0
1140,16913138352969222183,1058,2022-2023,NaN,NaN,True,Неуд,0,1.0,Белов Александр Владимирович,...,NaN,NaN,БИТ193,Таранухин Иван Денисович,0.0,0.0,NaN,NaN,5.0,25.0
1146,11856352064718161377,1347,2022-2023,ДЭИ,Прогр.,False,Уд,1,5.0,Якименко Святослав Игоревич,...,NaN,NaN,БПМ193,Тихонов Евгений Алексеевич,0.0,0.0,0.0,0.0,4.0,18.0
1150,15308223963626795785,831,2022-2023,NaN,NaN,False,Отл,1,9.0,Миколаенко Вадим Витальевич,...,0.527072,1.000000,БПМ213,Ткаченко Никита Андреевич,1.0,0.0,NaN,NaN,2.0,7.0
1154,10798765280746763851,1220,2022-2023,ДЭИ,Прогр-аппарат.,False,Отл,1,8.0,Королев Павел Сергеевич,...,NaN,NaN,МИВ221,Уркунов Айвар Кайратович,0.0,0.0,NaN,NaN,2.0,13.0


In [ ]:
preds

array([[3]])

In [ ]:
result

'\nСтудент: 15985224324041060664\nПредсказанный класс: 3 (вероятность: 0.44)\n Важные признаки:\n  • count_head_defenses = 21.0 (вклад: +0.2640)\n  • card_completed_total_wekan = 104.0 (вклад: +0.2210)\n  • avg_dt_between_commits_git = 0.6232767355933375 (вклад: +0.0776)\n  • count_head_project = 4.0 (вклад: +0.0680)\n  • student_commits_contribution_git = 0.863013698630137 (вклад: +0.0674)\n  • student_operation_contribution_git = 0.108018559277705 (вклад: +0.0469)\n  • hours_credited_wekan = 2.0 (вклад: -0.0435)\n'

In [ ]:
df

,person_id,project_id,year,department,type,isna,target_multiclass,target,rate_total,project_head,...,avg_dt_between_commits_git,student_commits_contribution_git,student_group,student_name,messages_updated_zulip,reactions_removed_zulip,hours_last_month_user_taiga,hours_last_year_user_taiga,count_head_project,count_head_defenses
0,15985224324041060664,1290,2022-2023,ДКИ,Прогр.,0,Хор,1,7.0,Зунин Владимир Викторович,...,0.623277,0.863014,БИВ205,Авдеюк Степан Сергеевич,0.0,0.0,None,None,4.0,21.0


## Сборка графа

In [ ]:
graph = StateGraph(ReasoningState)

graph.add_node("🤔 think", reason)
graph.add_node("1️⃣ First step", first_step)
graph.add_node("👨 write", answer)
graph.add_node("👨‍⚖️ critique", critique)
graph.add_node("🔍 search", search)
graph.add_node("🏁 finalize", finalize)

graph.add_edge(START, "🤔 think")
graph.add_edge("🤔 think", "1️⃣ First step")
graph.add_edge("👨 write", "👨‍⚖️ critique")
graph.add_edge("🔍 search", "👨 write")
graph.add_edge("🏁 finalize", END)



app = graph.compile()

# Запуск генерации

In [ ]:
with open("graph.mmd", "w") as f:
    f.write(app.get_graph().draw_mermaid())

In [ ]:
app.get_graph().draw_mermaid()

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\t__think(🤔 think)\n\t1___First_step(1️⃣ First step)\n\t__write(👨 write)\n\t_____critique(👨\u200d⚖️ critique)\n\t__search(🔍 search)\n\t__finalize(🏁 finalize)\n\t__end__([<p>__end__</p>]):::last\n\t1___First_step -.-> __finalize;\n\t1___First_step -.-> __write;\n\t1___First_step -.-> __search;\n\t__start__ --> __think;\n\t__write --> _____critique;\n\t_____critique -.-> __end__;\n\t_____critique -.-> __finalize;\n\t_____critique -.-> __search;\n\t__search --> __write;\n\t__think --> 1___First_step;\n\t__finalize --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 104.8 MB/s eta 0:00:00


In [ ]:
df[df.student_name == 'Иванов Иван Иванович']

,person_id,project_id,year,department,type,isna,target_multiclass,target,rate_total,project_head,...,avg_dt_between_commits_git,student_commits_contribution_git,student_group,student_name,messages_updated_zulip,reactions_removed_zulip,hours_last_month_user_taiga,hours_last_year_user_taiga,count_head_project,count_head_defenses


In [ ]:
import gradio as gr


def chat_with_assistant(user_input):
    state = {"user_question": user_input}
    try:
        result = app.invoke(state)
        return result.get("final_answer", "Извините, ответа нет.")
    except Exception as e:
        return f"Ошибка: {str(e)}"

# Интерфейс
with gr.Blocks() as demo:
    gr.Markdown("# 🎓 Университетский помощник")
    gr.Markdown("Введите вопрос, связанный с защитой проектов, и получите ответ.")

    with gr.Row():
        txt = gr.Textbox(label="Ваш вопрос")
        btn = gr.Button("Отправить")

    out = gr.Textbox(label="Ответ помощника", lines=6)

    btn.click(fn=chat_with_assistant, inputs=txt, outputs=out)
    txt.submit(fn=chat_with_assistant, inputs=txt, outputs=out)

# Запуск
demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a50e57756e7fe6b933.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
a = 'какую оценку получит студент Иванов Иван Иванович из группы БИВ216 за проект 19102'
# a.is

In [ ]:
# final 4
import uuid

b = 'расскажи, какой ты бот'
inputs = {"user_question": b}
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

start = time.time()
async for event in app.astream_events(inputs, version="v2"):
  event_type = event.get('event', None)
  agent = event.get('name', '')
  if agent in ["_write", "RunnableSequence", "__start__", "__end__", "LangGraph"]:
    continue
  if event_type == 'on_chat_model_stream':
    print(event['data']['chunk'].content, end='')
  elif event_type == 'on_chain_start':
    print(f"<{agent}>")
  elif event_type == 'on_chain_end':
    print(f"</{agent}>")
  # else:
   #print(event)

<🤔 think>
Вопрос пользователя касается моего предназначения как ИИ-помощника для преподавателей и сотрудников МИЭМ НИУ ВШЭ. Пользователь хочет понять, что я могу делать и как я могу помочь в контексте анализа данных и предсказания оценок студентов. 

**Мысли:**
1. **Целевая аудитория:** Я создан для помощи преподавателям и сотрудникам МИЭМ НИУ ВШЭ, которые занимаются управлением проектами студентов и их успеваемостью. Поэтому важно подчеркнуть мою способность работать с данными, связанными с активностью студентов и предоставлять точные прогнозы.
2. **Аналитика и поддержка:** Упор нужно сделать на том, что я помогаю анализировать данные и предлагать вероятные оценки на основе анализа активности студента, выполнения задач и других факторов. Это может быть полезно для более объективного оценивания работы студентов.
3. **Ограничения:** Важно отметить, что я не могу обсуждать темы, не связанные с образовательным процессом, и не даю общих рекомендаций по программированию или коду. Также я не

In [ ]:
# final 3 HERE HERE HERE!!!
import uuid

inputs = {"user_question": a}
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

start = time.time()
async for event in app.astream_events(inputs, version="v2"):
  event_type = event.get('event', None)
  agent = event.get('name', '')
  if agent in ["_write", "RunnableSequence", "__start__", "__end__", "LangGraph"]:
    continue
  if event_type == 'on_chat_model_stream':
    print(event['data']['chunk'].content, end='')
  elif event_type == 'on_chain_start':
    print(f"<{agent}>")
  elif event_type == 'on_chain_end':
    print(f"</{agent}>")
  # else:
   #print(event)

<🤔 think>
Ответ на этот запрос требует использования нескольких источников данных: информации о деятельности студента в различных цифровых платформах (например, активность в чатах, трекерах задач, репозиториях кода), а также оценок за предыдущие проекты или курсовые работы. На основании этих данных можно построить прогнозную модель, которая позволит оценить вероятность получения той или иной оценки.

**Почему пользователь задает такой вопрос?**
Этот вопрос может быть вызван несколькими причинами:
1. **Заинтересованность в успехе студента** – родитель или сам студент хочет знать, насколько хорошо он справился с проектом, чтобы скорректировать свою дальнейшую работу или мотивировать себя.
2. **Административная необходимость** – это может быть связано с внутренними процессами университета, например, если нужно утвердить результаты проекта до окончания семестра.
3. **Совместная работа над проектом** – возможна ситуация, когда несколько студентов работают вместе, и требуется уточнить вклад 

<ipython-input-49-71ca2e76ea3d>:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_class = int(preds[i])


Исходя из предоставленных данных, вероятность того, что студент Иванов Иван Иванович получит оценку "хорошо" за проект 19102, составляет **0.44**. Это значение является результатом анализа различных факторов, таких как количество защит научных руководителей, среднее время выполнения задач в проекте и другие показатели.</👨 write>
<👨‍⚖️ critique>
```json
{
  "thoughts": "<THINK>\nВаш запрос требует интеграции данных из нескольких источников, включая систему управления проектами, базы данных оценок и активности студента. Также необходимо использовать прогностические модели для оценки успеваемости студента. Прогнозирование успеха студента требует анализа различных факторов, таких как активность в чатах, трекерах задач, репозиториях кода и других цифровых платформ.\n\nПричина, по которой вы спрашиваете, может заключаться в заинтересованности в успехе студента, административной необходимости или совместной работе над проектом. Важно подчеркнуть все аспекты работы студента, а не только конечн

Вопрос пользователя касается моего предназначения как ИИ-помощника для преподавателей и сотрудников МИЭМ НИУ ВШЭ. Пользователь хочет понять, что я могу делать и как я могу помочь в контексте анализа данных и предсказания оценок студентов.

**Мысли:**
1. **Целевая аудитория:** Я создан для помощи преподавателям и сотрудникам МИЭМ НИУ ВШЭ, которые занимаются управлением проектами студентов и их успеваемостью. Поэтому важно подчеркнуть мою способность работать с данными, связанными с активностью студентов и предоставлять точные прогнозы.
2. **Аналитика и поддержка:** Упор нужно сделать на том, что я помогаю анализировать данные и предлагать вероятные оценки на основе анализа активности студента, выполнения задач и других факторов. Это может быть полезно для более объективного оценивания работы студентов.
3. **Ограничения:** Важно отметить, что я не могу обсуждать темы, не связанные с образовательным процессом, и не даю общих рекомендаций по программированию или коду. Также я не придумываю данные, а работаю исключительно с существующими фактами.
4. **Профессионализм:** Нужно показать, что мои ответы всегда будут четкими и релевантными, чтобы помочь пользователям сосредоточиться на вопросах, связанных с учебным процессом.

**Ответ:**
Я являюсь ИИ-помощником, специально обученным для работы с данными студентов МИЭМ НИУ ВШЭ. Моя основная функция заключается в анализе активности студентов на цифровых платформах, таких как Zulip, Wekan, Taiga и Git, а также в использовании этих данных для предсказания возможных оценок за проекты. Я могу предоставить детальный анализ активности студента, включая его участие в обсуждениях, выполнение задач и прогресс в проекте. Этот анализ помогает преподавателям принимать более обоснованные решения при выставлении оценок. Я не занимаюсь общими вопросами или темами, не связанными с учебным процессом, и не создаю новые данные – вся информация основана на реальных фактах.


In [ ]:
# final 2
import uuid

inputs = {"user_question": a}
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

start = time.time()
async for event in app.astream_events(inputs, version="v2"):
  event_type = event.get('event', None)
  agent = event.get('name', '')
  if agent in ["_write", "RunnableSequence", "__start__", "__end__", "LangGraph"]:
    continue
  if event_type == 'on_chat_model_stream':
    print(event['data']['chunk'].content, end='')
  elif event_type == 'on_chain_start':
    print(f"<{agent}>")
  elif event_type == 'on_chain_end':
    print(f"</{agent}>")
  # else:
   #print(event)

<🤔 think>
Я понимаю, что запрос касается оценки студента за проект, основываясь на его цифровой активности и данных из базы данных. Однако важно помнить, что такая информация должна использоваться исключительно в образовательных целях и не может быть передана без согласия самого студента. Кроме того, необходимо соблюдать правила конфиденциальности и этические нормы.

Чтобы ответить на этот вопрос, я могу:

1. **Обратиться к истории активности студента**: Посмотреть его участие в чатах, трекерах задач, репозиториях и других платформах. Это поможет понять уровень его вовлеченности и выполнения задач.
   
2. **Предсказать оценку на основе предыдущих достижений**: Если у нас есть модель прогнозирования оценок, можно использовать её для расчета вероятности получения той или иной оценки.

3. **Проанализировать соответствие работы стандартам**: Оценить качество кода, своевременность сдачи задач и наличие ошибок.

4. **Учесть вклад студента в команду**: Иногда важны не только индивидуальные до

<ipython-input-49-71ca2e76ea3d>:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_class = int(preds[i])


Из предоставленных данных видно, что вероятность получения студентом Ивановым Иваном Ивановичем хорошей оценки ("хорошо") составляет **0.44**. Однако важно учитывать, что это лишь одна из возможных оценок, и на итоговую оценку могут влиять и другие факторы, такие как общая активность студента, качество подготовки к защите проекта, взаимодействие с научным руководителем и другими участниками команды. Для более точного прогноза необходимо проанализировать дополнительные аспекты работы студента над проектом.</👨 write>
<👨‍⚖️ critique>
```json
{
  "thoughts": "<THINK>",
  "critique": "<OLD_CRITIQUE>",
  "search_query": "",
  "final_decision": "good"
}
```</👨‍⚖️ critique>
<🏁 finalize>
Исходя из анализа ваших данных и активности, вероятность получения вами оценки «хорошо» за проект составляет **44%**. Важно отметить, что эта оценка основана на текущей информации, и на нее могут повлиять ваши усилия в оставшееся до завершения проекта время, подготовка к защите и взаимодействие с научным руково

In [ ]:
# final 0
import uuid

inputs = {"user_question": a}
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

start = time.time()
async for event in app.astream_events(inputs, version="v2"):
  event_type = event.get('event', None)
  agent = event.get('name', '')
  if agent in ["_write", "RunnableSequence", "__start__", "__end__", "LangGraph"]:
    continue
  if event_type == 'on_chat_model_stream':
    print(event['data']['chunk'].content, end='')
  elif event_type == 'on_chain_start':
    print(f"<{agent}>")
  elif event_type == 'on_chain_end':
    print(f"</{agent}>")
  # else:
   #print(event)

<🤔 think>
Я понимаю запрос и могу предоставить информацию на основе данных, которые доступны мне из цифрового следа студента. Однако важно помнить, что ответ будет основан исключительно на данных, а не на субъективных оценках или мнениях.

**Мысли о вопросе:**

1. **Зачем студент спрашивает об оценке?** Если это студент, то он может быть заинтересован в своих результатах или ожидает подсказки для улучшения работы над проектом.
   
2. **Что нужно учитывать при ответе?** Оценка формируется на основе множества факторов, таких как участие в обсуждениях, выполнение задач, качество кода, своевременность сдачи работ и т. д. Поэтому, чтобы дать точный прогноз, мне нужно иметь доступ к полному набору данных по студенту.

3. **Ограничения ответа:** Я не могу делать предположения или прогнозы на основе общих суждений или необоснованных данных. Моя роль — предоставлять ответы, основанные на реальных событиях и информации, которая доступна в системе.

**Ответ на вопрос:**

Я могу сообщить вам, каку

<ipython-input-49-71ca2e76ea3d>:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_class = int(preds[i])


Для того чтобы дать точный прогноз оценки студента Ивана Ивановича Иванова из группы БИВ216 за проект с номером 19102, необходимо проанализировать предоставленные вами данные. Из результатов поиска видно, что ключевыми факторами, влияющими на итоговую оценку, являются следующие показатели:

1. **Количество защит студентов в год у научного руководителя**: 48.0 защит (вклад: +0.1572). Это указывает на то, что чем больше защит у научного руководителя, тем выше вероятность получения высокой оценки.

2. **Максимальное количество часов на одну задачу в проекте**: 160.0 часов (вклад: +0.1017). Чем меньше времени тратится на выполнение задачи, тем лучше результат.

3. **Количество проектов, которые ведет научный руководитель**: 8.0 проектов (вклад: +0.0772). Здесь работает тот же принцип, что и в пункте 1 – большее количество проектов повышает шансы на высокую оценку.

4. **Среднее время выполнения задач в проекте**: 14.2 часа (вклад: +0.0712). Аналогично, меньшее время выполнения задач положи

<ipython-input-49-71ca2e76ea3d>:86: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_class = int(preds[i])


Исходя из предоставленных данных, вероятность того, что студент Иванов Иван Иванович получит оценку "хорошо" за проект 19102, составляет **0.44**. Однако стоит учесть, что этот результат основан на анализе ряда факторов, таких как количество защит научных руководителей, среднее время выполнения задач, средняя активность студента и другие показатели. Эти данные помогают оценить вклад различных аспектов в итоговую оценку, но окончательное решение может зависеть от других, не учтенных здесь критериев.

Пожалуйста, обратите внимание, что эти результаты являются прогнозами и могут изменяться в зависимости от дополнительных обстоятельств или требований учебного заведения.</👨 write>
<👨‍⚖️ critique>
```json
{
  "thoughts": "<THINK>\nЯ понимаю запрос и могу предоставить информацию на основе данных, которые доступны мне из цифрового следа студента. Однако важно помнить, что ответ будет основан исключительно на данных, а не на субъективных оценках или мнениях.\n\n**Мысли о вопросе:**\n1. **Зачем 

# Подключение Arize Phoenix

In [ ]:
!pip install arize-phoenix opentelemetry-exporter-otlp openinference-instrumentation-langchain -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.2 MB/s eta 0:00:00


In [ ]:
import phoenix as px
session = px.launch_app(use_temp_dir=False) # Fo colab
# session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://h3mds7xiijm3-496ff2e9c6d22116-6006-colab.googleusercontent.com/
💽 Your data is being persisted to sqlite:////root/.phoenix/phoenix.db
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
from phoenix.otel import register
from openinference.instrumentation.langchain import LangChainInstrumentor

tracer_provider = register(
  project_name="Agentic reasoning", # Default is 'default'
  endpoint="http://localhost:4317",
)

LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: Agentic reasoning
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

